In [1]:
import  requests as re
from bs4 import BeautifulSoup
from pandas import DataFrame, Series
import numpy as np
import pandas as pd
import csv


def read_excel():
    f = open('kospi_stock_code.csv', 'r', encoding='utf-8')
    rdr = csv.reader(f)
    codes = {}
    lim = 1
    for line in rdr:
        codes[line[1]] = line[2]
        lim= lim+1
        if lim==30:
            break
    f.close()
    return codes

def calculate_price(new_t):
    new_BPS = new_t.loc['BPS'][-1]
    new_price = new_BPS/((1.15)**8)
    return new_price
    
#print(read_excel())
tickers = {'005490' : '포스코',
          '123890' : '한국자산신탁'} 
#read_excel()

#my_folder = "C:/Users/user/Desktop/stock"
ap = 1
for code in tickers.keys() :
    print(ap)
    """재무재표"""
    print(tickers[code])
    url = re.get('http://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A%s' % (code))
    url = url.content
    html = BeautifulSoup(url,'html.parser')
    
    body = html.find('body')
    if(not body):
        continue
    fn_body = body.find('div',{'class':'fng_body asp_body'})
    ur_table = fn_body.find('div',{'id': 'div15'})
    table = ur_table.find('div',{'id':'highlight_D_Y'})
    tbody = table.find('tbody')
    tr = tbody.find_all('tr')
    Table = DataFrame()
        
    price = body.find('span',{'id':'svdMainChartTxt11'})
    price = price.text
        
    around_PER = body.find('div',{'id': 'corp_group2'}).find_all('dd')
    dd_list = []
    for tag in around_PER:
        dd_list.append(tag.text)
    around_PER = dd_list[5]
        
    print("현재 주가 :" + price)
    print("업종 PER : "+around_PER)   

    for i in tr:
        #항목 가져오기
        category = i.find('span',{'class':'txt_acd'})

        if category == None:
            category = i.find('th')
        category = category.text.strip()
            #값 가져오기
        value_list = []
        j = i.find_all('td',{'class','r'})

        for value in j:
            temp = value.text.replace(',','').strip()
            try:
                temp = float(temp)
                value_list.append(temp)
            except:
                value_list.append(0)
        Table['%s'%(category)] = value_list

            #기간 가져오기
        thead = table.find('thead')
        tr_2 = thead.find('tr',{'class','td_gapcolor2'}).find_all('th')
        year_list = []
        for i in tr_2:
            try:
                temp_year = i.find('span', {'class','txt_acd'}).text
            except:
                temp_year = i.text
                
            year_list.append(temp_year)
            #del year_list[-1]
        Table.index = year_list
    new_t = Table.T.iloc[[16,17,18,19,21]]
    ROA_avg = new_t.mean(axis=1)[0]
    ROE_avg = new_t.mean(axis=1)[1]
    bf_num = -2
        
    new_column = []
    for i in range(0,6):
        new_year = 2023+i
        new_t['%d'%new_year] = None
        for j in range(2,6):
            
            if(new_t.loc['BPS'][-j] != 0):
                break
            else:
                bf_num = bf_num-1
        new_EPS = new_t.loc['BPS'][bf_num]*(ROE_avg/100)
        new_BPS = new_t.loc['BPS'][bf_num] + new_EPS
        bf_num = -2
            
        new_t['%d'%new_year] = ROA_avg,ROE_avg,new_EPS, new_BPS, 'NAN'
            
        #new_t.to_csv('%s/%s.csv'%(my_folder,tickers[code]))
    display(new_t)
    ap = ap+1
    print("현재가치 : " ,round(calculate_price(new_t),2))    






1
포스코
현재 주가 :164,500
업종 PER : 11.11


,2015/12,2016/12,2017/12,2018/12,2019/12,2020/12(E),2021/12(E),2022/12(E),2023,2024,2025,2026,2027,2028
ROA,-0.12,1.31,3.75,2.41,2.52,2.43,3.21,3.38,2.36125,2.36125,2.36125,2.36125,2.36125,2.36125
ROE,0.44,3.26,6.48,3.88,4.18,3.95,5.24,5.38,4.10125,4.10125,4.10125,4.10125,4.10125,4.10125
EPS,2072.00,15637.00,32001.00,19391.00,21048.00,20355.00,27744.00,29438.00,23511.8,24476,25479.9,26524.9,27612.7,28745.2
BPS,490547.00,503596.00,519183.00,515032.00,527375.00,538134.00,554738.00,573283.00,596795,621271,646751,673276,700888,729633
PER,80.36,16.47,10.39,12.53,11.24,8.08,5.93,5.59,NAN,NAN,NAN,NAN,NAN,NAN


현재가치 :  238518.47
2
한국자산신탁
현재 주가 :2,270
업종 PER : 5.87


,2015/12,2016/12,2017/12,2018/12,2019/12,2020/12(E),2021/12(E),2022/12(E),2023,2024,2025,2026,2027,2028
ROA,13.27,15.94,18.81,11.57,7.98,7.85,7.25,0.0,10.3337,10.3337,10.3337,10.3337,10.3337,10.3337
ROE,20.57,22.59,27.12,18.52,14.28,14.21,12.41,0.0,16.2125,16.2125,16.2125,16.2125,16.2125,16.2125
EPS,406.00,631.00,1013.00,826.00,723.00,824.00,827.00,0.0,1163.9,1352.59,1571.88,1826.72,2122.88,2467.05
BPS,2153.00,3337.00,4210.00,4886.00,5441.00,6352.00,7179.00,0.0,8342.9,9695.49,11267.4,13094.1,15217,17684
PER,0.00,9.93,6.05,4.85,4.66,2.76,2.75,0.0,NAN,NAN,NAN,NAN,NAN,NAN


현재가치 :  5780.94


In [20]:
read_excel()


{'060310': '3S',
 '095570': 'AJ네트웍스',
 '006840': 'AK홀딩스',
 '054620': 'APS홀딩스',
 '265520': 'AP시스템',
 '211270': 'AP위성',
 '027410': 'BGF',
 '282330': 'BGF리테일',
 '138930': 'BNK금융지주'}